# <center>ZUM</center>
# <center> Projekt -- dokumentacja końcowa </center>

## <center> Jan Budziński -- 310609 </center>
## <center> Jarosław Nachyła -- nr_indeksu </center>

## 2. Implementacja

### 2.1. Nienadzorowana detekcja anomalii

W ramach projektu zaimplementowano klasę <i>AnomalyDetector</i>, będącą opakowaniem na algorytmy grupowania KMeans, DBSCAN oraz AgglomerativeClustering, jak również na miary niepodobieństwa euklidesową, Mahalanobisa i Manhattan. Celem tejże klasy jest stworzenie łatwego w obsłudze środowiska testowego, które w miarę możliwości (niektóre algorytmy różnią się wejściami) będzie jednorodne w użyciu dla każdego z porównywanych algorytmów i miar.
Klasa ta zawiera następujące metody:

- fit -- dopasowuje model do danych wejściowych,
- fit_predict -- wykonuje trening modelu i jednocześnie przewidująca klasy i określająca dystanse od centrów klastrów dla danych wejściowych,
- transform_distances -- określa, czy dane wejściowe są anomaliami na podstawie dystansu otrzymanego z wybranej miary niepodobieństwa,
- transform_labels -- dla liczby klastrów większych niż 2, metoda ta zmienia przypisanie do wszystkich klastrów poza najliczniejszym w anomalię.


Dzięki tym metodom użytkownik jest w stanie w łatwy sposób testować różne modele i miary niepodobieństwa, zmieniając wyłącznie jeden parametr w kodzie.

Kod implementujący tę klasę jest w pliku anomaly_detector.py.

Ponadto utworzono klasę <i>AnomalyDetectorEvaluator</i> zawierającą metody obliczające metryki potrzebne do ewaluacji wytrenowanych modeli.

Testowane metryki to:

- dokładność (accuracy)
- precyzja (precision)
- czułość (recall)
- pole pod wykresem PRC

Wszystkie te metryki zostały dokładniej opisane w dokumentacji wstępnej. Jednakże, z uwagi na fakt, iż dane z założenia są wysoce niezbalansowane (jako że są to dane anomalii, to wejścia o pozytywnej klasie stanowią poniżej 1% wszystkich) uznano, że zwykła dokładność może niewiele powiedzieć, jako że przypisanie wszystkim danym klasy negatywnej pozwala osiągnąć powyżej 99% poprawnych predykcji. W tym celu dodano metrykę dokładności wykrywania outlierów, która testuje, ile spośród prawdziwych outlierów zostało poprawnie zidentyfikowanych. 

Kod z klasą AnomalyDetectorEvaluator znajduje się w pliku metrics.py.

### 2.2. Jednoklasowy klasyfikator

## 3. Eksperymenty



In [ ]:
import os
import json
import pandas as pd

def load_metrics_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

def process_files(directory):
    results = []
    
    # Przeglądanie wszystkich plików w katalogu
    for file_name in os.listdir(directory):
        if file_name.endswith(".json"):
            # Parsing nazwy pliku
            parts = file_name.split('_')
            model = parts[0]
            dataset = parts[1]
            metric = parts[2].split('.')[0]

            file_path = os.path.join(directory, file_name)
            metrics_data = load_metrics_from_json(file_path)

            for entry in metrics_data:
                avg_metrics = entry['avg_metrics']
                result = {
                    'model': model,
                    'dataset': dataset,
                    'metric': metric,
                    'accuracy': avg_metrics.get('accuracy'),
                    'precision': avg_metrics.get('precision'),
                    'recall': avg_metrics.get('recall'),
                    'f1': avg_metrics.get('f1'),
                    'positive_recall': avg_metrics.get('positive_recall'),
                    'negative_recall': avg_metrics.get('negative_recall'),
                    'positive_precision': avg_metrics.get('positive_precision'),
                    'negative_precision': avg_metrics.get('negative_precision'),
                    'auc_score': avg_metrics.get('auc_pr')
                }
                results.append(result)
    
    # Tworzenie DataFrame z wynikami
    df = pd.DataFrame(results)
    return df

### 3.1 Porównanie modeli grupowania

W ramach pierwszego eksperymentu wykonano testy porównawcze iloczynu kartezjańskiego 3 wybranych modeli, 3 miar niepodobieństwa i 2 zbiorów danych, czyli łącznie 18 pojedynczych testów.

Podczas przeprowadzania eksperymentu wystąpiły pewne problemy. Mianowicie, algorytm AgglomerativeClustering cechuje się wysokimi złożonościami: złożoność obliczeniowa to $O(n^2)$, a czasowa to $O(n^3)$. Z tego względu trening na całych zbiorach danych, a w szczególności na zbiorze HTTP był niemożliwy. W związku z tym zastosowano subsampling.

Wyniki testów przedstawione są w poniższych tabelach.



In [4]:
import os

import pandas as pd

def load_metrics_from_json(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data
def process_files(directory, pdataset = "http"):
    results = []

    # Przeglądanie wszystkich plików w katalogu
    for file_name in os.listdir(directory):
        if file_name.endswith(".json"):
            # Parsing nazwy pliku
            parts = file_name.split('_')
            model = parts[0]
            dataset = parts[1]

            if dataset != pdataset:
                continue



            file_path = os.path.join(directory, file_name)
            metrics_data = load_metrics_from_json(file_path)



            if isinstance(metrics_data, dict):  # Dla isolation forest i svm
                metric = "N/A"
                avg_metrics = metrics_data['avg_metrics']
                result = {
                    'model': model,
                    'dataset': dataset,
                    'metric': metric,
                    'accuracy': avg_metrics.get('accuracy'),
                    'precision': avg_metrics.get('precision'),
                    'recall': avg_metrics.get('recall'),
                    'f1': avg_metrics.get('f1'),
                    'positive_recall': avg_metrics.get('positive_recall'),
                    'negative_recall': avg_metrics.get('negative_recall'),
                    'positive_precision': avg_metrics.get('positive_precision'),
                    'negative_precision': avg_metrics.get('negative_precision'),
                    'auc_score': avg_metrics.get('auc_pr')
                }
                results.append(result)
            else:  # Dla pozostałych modeli
                for entry in metrics_data:
                    metric = entry['metric']
                    avg_metrics = entry['avg_metrics']
                    result = {
                        'model': model,
                        'dataset': dataset,
                        'metric': metric,
                        'accuracy': avg_metrics.get('accuracy'),
                        'precision': avg_metrics.get('precision'),
                        'recall': avg_metrics.get('recall'),
                        'f1': avg_metrics.get('f1'),
                        'positive_recall': avg_metrics.get('positive_recall'),
                        'negative_recall': avg_metrics.get('negative_recall'),
                        'positive_precision': avg_metrics.get('positive_precision'),
                        'negative_precision': avg_metrics.get('negative_precision'),
                        'auc_score': avg_metrics.get('auc_pr')
                    }
                    results.append(result)

    # Tworzenie DataFrame z wynikami
    df = pd.DataFrame(results)
    return df

process_files('./')

,model,dataset,metric,accuracy,precision,recall,f1,positive_recall,negative_recall,positive_precision,negative_precision,auc_score
0,isolationforest,http,N/A,0.907282,0.729462,1.000000,0.843571,1.000000,0.876376,0.729462,1.000000,NaN
1,dbscan,http,euclidean,0.993616,0.854988,1.000000,0.921826,1.000000,0.993366,0.854988,1.000000,0.035111
2,dbscan,http,manhattan,0.993616,0.854988,1.000000,0.921826,1.000000,0.993366,0.854988,1.000000,0.035111
3,svm,http,N/A,0.623021,0.398738,1.000000,0.570139,1.000000,0.497362,0.398738,1.000000,NaN
4,metacost,http,mahalanobis,0.580502,0.007331,0.794211,0.014528,0.794211,0.579666,0.007331,0.998617,NaN
5,metacost,http,euclidean,0.747444,0.036003,0.017910,0.012120,0.017910,0.750297,0.036003,0.994549,NaN
6,metacost,http,cityblock,0.570846,0.008953,0.995025,0.017747,0.995025,0.569187,0.008953,0.999966,NaN
7,kmeans,http,euclidean,0.565949,0.008857,0.995477,0.017557,0.995477,0.564269,0.008857,0.999969,0.538962
8,kmeans,http,cityblock,0.999942,0.988779,0.996382,0.992566,0.996382,0.999956,0.988779,0.999986,0.015064
9,kmeans,http,mahalanobis,0.990934,0.011000,0.014925,0.012666,0.014925,0.994751,0.011000,0.996142,0.538960


In [5]:
process_files('./', pdataset="shuttle")

,model,dataset,metric,accuracy,precision,recall,f1,positive_recall,negative_recall,positive_precision,negative_precision,auc_score
0,dbscan,shuttle,euclidean,0.993971,0.957326,0.958416,0.957871,0.958416,0.996710,0.957326,0.996797,0.215972
1,dbscan,shuttle,manhattan,0.995376,0.954596,0.982056,0.968131,0.982056,0.996402,0.954596,0.998615,0.216555
2,svm,shuttle,N/A,0.635503,0.406425,0.994588,0.577047,0.994588,0.515807,0.406425,0.996515,NaN
3,agglomerative,shuttle,euclidean,0.995988,0.999096,0.944745,0.971161,0.944745,0.999934,0.999096,0.995762,0.435803
4,agglomerative,shuttle,manhattan,0.995988,0.999096,0.944745,0.971161,0.944745,0.999934,0.999096,0.995762,0.530860
5,agglomerative,shuttle,mahalanobis,0.995988,0.999096,0.944745,0.971161,0.944745,0.999934,0.999096,0.995762,0.429757
6,kmeans,shuttle,euclidean,0.974907,0.943903,0.690117,0.797302,0.690117,0.996841,0.943903,0.976617,0.373327
7,kmeans,shuttle,manhattan,0.786851,0.008621,0.017374,0.011524,0.017374,0.846115,0.008621,0.917898,0.845926
8,kmeans,shuttle,mahalanobis,0.996089,0.999098,0.946169,0.971913,0.946169,0.999934,0.999098,0.995871,0.429757
9,isolationforest,shuttle,N/A,0.888208,0.693906,0.989177,0.815641,0.989177,0.854552,0.693906,0.995796,NaN
